In [6]:
from yahoofinancials import YahooFinancials
from http.client import IncompleteRead
import csv
import pandas as pd

Read all FIN companies from txt file

In [7]:
with open('companies/FIN.txt') as csvfile:
    data = csv.reader(csvfile, delimiter=',')
    FIN=[i for row in data for i in row]
print(len(FIN))

177


### Function to get data from YahooFinancials

In [10]:
def company_data():
    max_try = 5
    for N in range(len(FIN)):
        attemp = 0
        while attemp < max_try:
            try:
                yf = YahooFinancials(FIN[N])
                
                income_statement = yf.get_financial_stmts('annual', 'income')
                company_is = income_statement['incomeStatementHistory'][FIN[N]]
                
                balance_statement = yf.get_financial_stmts('annual', 'balance')
                company_bs = balance_statement['balanceSheetHistory'][FIN[N]]
                
                cash_flow = yf.get_financial_stmts('annual', 'cash')
                company_cf = cash_flow['cashflowStatementHistory'][FIN[N]]

                df_list = []
                for d in company_is:
                    df_list.append(pd.DataFrame.from_dict(d, orient='index'))   
                df_is = pd.concat(df_list)
                df_list.clear()

                for d in company_bs:
                    df_list.append(pd.DataFrame.from_dict(d, orient='index'))   
                df_bs = pd.concat(df_list)
                df_list.clear()

                for d in company_cf:
                    df_list.append(pd.DataFrame.from_dict(d, orient='index'))   
                df_cf = pd.concat(df_list)

                df = pd.concat([df_is, df_bs, df_cf], axis=1)
                df['company'] = FIN[N]
                df.to_csv(f"companies/FIN_{N}.csv", index=1,  index_label='date')

                print(f"FIN_{N}.csv")

            except IncompleteRead:
                attemp += 1

                if attemp == max_try:
                    print(f'Failed {N}')
            except ValueError as e:
                print(f'{e}: {N}')
                attemp = max_try
            else:
                break

In [9]:
company_data()

FIN_0.csv


### Combine all companies to 1 csv file

In [ ]:
companies = []

for n in range(len(FIN)):
    try:
        company = pd.read_csv(f'companies/FIN_{n}.csv')
        companies.append(company)
    except FileNotFoundError:
        pass

df = pd.concat(companies)
df.to_csv("companies/FIN_all.csv", index=False)
df